In [1]:
!pip install lightgbm

You should consider upgrading via the 'C:\Users\elousiren\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install catboost

You should consider upgrading via the 'C:\Users\elousiren\anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
from scipy.stats import probplot, ks_2samp
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, auc
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import xgboost as xgb
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder
from copy import deepcopy
import catboost as cb
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from typing import List, Tuple, Optional
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
from sklearn.decomposition import PCA
from sklearn.model_selection import GroupKFold
import math 
import re 

In [4]:
train = pd.read_csv("C:/Users/elousiren/Desktop/git_presentation/playground/gb/sport/3/assignment_2_train.csv")
test = pd.read_csv("C:/Users/elousiren/Desktop/git_presentation/playground/gb/sport/3/assignment_2_test.csv")

In [5]:
train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
test.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,3287000,1,7415038,226.00,W,12473,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3287001,0,7415054,3072.00,W,15651,417.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3287002,0,7415081,319.95,W,13844,583.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3287003,0,7415111,171.00,W,11556,309.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3287004,0,7415112,107.95,W,10985,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
y_train = train['isFraud'] 
y_test = test['isFraud'] 
x_train = train.drop('isFraud',axis=1)
x_test = test.drop('isFraud',axis=1)
cat_columns = list(x_test.select_dtypes(include=[np.object]))
for c in cat_columns:
    x_train[c] = x_train[c].fillna('missing').astype('category')
    x_test[c] = x_test[c].fillna('missing').astype('category')

### Задание 0: выбрать любую модель машнного обучения и зафиксировать любой тип валидации. Обучить базовую модель и зафиксировать базовое качество модели. В каждом следующем задании нужно будет обучить выбранную модель и оценивать ее качество на зафиксированной схеме валидации. После каждого задания, требуется сделать вывод о достигаемом качестве модели, по сравнению с качестом из предыдущего шага.

In [8]:
x_valid, x_def = train_test_split(x_train, train_size=0.5, random_state=42)
y_valid, y_def = train_test_split(y_train, train_size=0.5, random_state=42)

In [9]:
params = {
    "n_estimators": 1000,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 10,
    "verbose": 50,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 100,
    "thread_count": 6,
    "random_seed": 42
}

In [10]:
cat_f = ['ProductCD','card4','card6','P_emaildomain','R_emaildomain','M1','M2','M3','M4','M5','M6','M7','M8','M9']
def cat_check():
    global x_train, y_train, x_valid, x_test, x_def, y_valid, y_test, y_def, cat_f
    cat_cat = cb.CatBoostClassifier(**params)
    cat_cat.fit(x_train, y_train,
               eval_set=[(x_train, y_train), (x_test, y_test)],
               cat_features = cat_f)

    train_score = roc_auc_score(y_train, cat_cat.predict_proba(x_train)[:,1])
    valid_score= roc_auc_score(y_valid, cat_cat.predict_proba(x_valid)[:,1])
    test_score= roc_auc_score(y_test, cat_cat.predict_proba(x_test)[:,1])
    def_score= roc_auc_score(y_def, cat_cat.predict_proba(x_def)[:,1])


    print(f"Train-score: {round(train_score, 3)}, Valid-score: {round(test_score, 3)}, отложенная выборка--score: {round(def_score, 3)}, Test-score(LD): {round(valid_score, 3)}")

In [11]:
cat_check()

0:	test: 0.6681376	test1: 0.6614100	best: 0.6614100 (0)	total: 861ms	remaining: 14m 20s
50:	test: 0.8479427	test1: 0.8269280	best: 0.8274266 (49)	total: 30.5s	remaining: 9m 27s
100:	test: 0.8676333	test1: 0.8409268	best: 0.8409268 (100)	total: 1m	remaining: 8m 54s
150:	test: 0.8786981	test1: 0.8452584	best: 0.8455145 (145)	total: 1m 29s	remaining: 8m 23s
200:	test: 0.8861514	test1: 0.8510249	best: 0.8511748 (199)	total: 1m 59s	remaining: 7m 53s
250:	test: 0.8904359	test1: 0.8549281	best: 0.8549281 (250)	total: 2m 27s	remaining: 7m 20s
300:	test: 0.8938660	test1: 0.8574695	best: 0.8574695 (300)	total: 2m 56s	remaining: 6m 49s
350:	test: 0.8967074	test1: 0.8590071	best: 0.8590071 (350)	total: 3m 25s	remaining: 6m 20s
400:	test: 0.8986406	test1: 0.8599462	best: 0.8599740 (392)	total: 3m 54s	remaining: 5m 50s
450:	test: 0.8998571	test1: 0.8604758	best: 0.8605001 (448)	total: 4m 22s	remaining: 5m 20s
500:	test: 0.9012979	test1: 0.8615548	best: 0.8615793 (494)	total: 4m 51s	remaining: 4m 50s

### Задание 1: признак TransactionDT - это смещение в секундах относительно базовой даты. Базовая дата - 2017-12-01, преобразовать признак TransactionDT в datetime, прибавив к базовой дате исходное значение признака. Из полученного признака выделить год, месяц, день недели, час, день.

In [12]:
def datamaker(df):
    df['TransactionDT'] = pd.to_datetime(df['TransactionDT'], unit ='s',origin='2017-12-01') 
    df['TransactionDT_year'] = df['TransactionDT']
    df['TransactionDT_month'] = df['TransactionDT'].dt.month
    df['TransactionDT_weekday'] = df['TransactionDT'].dt.weekday
    df['TransactionDT_year_hour'] = df['TransactionDT'].dt.hour
    df['TransactionDT_year_day'] = df['TransactionDT'].dt.day

In [13]:
datamaker(x_train)
datamaker(x_valid)
datamaker(x_test)
datamaker(x_def)

C:\Users\elousiren\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\elousiren\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\elousiren\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [14]:
cat_check()

0:	test: 0.6508269	test1: 0.6590844	best: 0.6590844 (0)	total: 618ms	remaining: 10m 17s
50:	test: 0.8492334	test1: 0.8289377	best: 0.8312913 (39)	total: 30.1s	remaining: 9m 20s
100:	test: 0.8698990	test1: 0.8410371	best: 0.8422936 (79)	total: 59.6s	remaining: 8m 50s
150:	test: 0.8786690	test1: 0.8420903	best: 0.8422936 (79)	total: 1m 29s	remaining: 8m 22s
200:	test: 0.8870419	test1: 0.8521060	best: 0.8521060 (200)	total: 1m 58s	remaining: 7m 52s
250:	test: 0.8911976	test1: 0.8548054	best: 0.8548095 (249)	total: 2m 27s	remaining: 7m 20s
300:	test: 0.8944086	test1: 0.8569943	best: 0.8569943 (300)	total: 2m 56s	remaining: 6m 50s
350:	test: 0.8978073	test1: 0.8591501	best: 0.8591501 (350)	total: 3m 26s	remaining: 6m 21s
400:	test: 0.8997935	test1: 0.8603193	best: 0.8603193 (400)	total: 3m 55s	remaining: 5m 51s
450:	test: 0.9011300	test1: 0.8611672	best: 0.8612258 (444)	total: 4m 24s	remaining: 5m 22s
500:	test: 0.9035148	test1: 0.8622636	best: 0.8622636 (500)	total: 4m 53s	remaining: 4m 52

### Задание 2: сделать конкатенацию признаков
* card1 + card2;
* card1 + card2 + card_3 + card_5;
* card1 + card2 + card_3 + card_5 + addr1 + addr2
Рассматривать их как категориальных признаки.

In [15]:
def featmaker(df):
    df['card1-2'] = df['card1'].fillna('').astype('string')+ df['card2'].fillna('').astype('string')
    df['card1-5'] = df['card1-2'] +df['card3'].fillna('').astype('string')+ df['card5'].fillna('').astype('string')
    df['card1-card5_addr1-2']= df['card1-5'].fillna('').astype('string') + df['addr1'].fillna('').astype('string') + df['addr2'].fillna('').astype('string')
    df['card1-2']=df['card1-2'].astype('category')
    df['card1-5'] = df['card1-5'].astype('category')
    df['card1-card5_addr1-2'] = df['card1-card5_addr1-2'].astype('category')

In [16]:
featmaker(x_train)
featmaker(x_valid)
featmaker(x_test)
featmaker(x_def)

C:\Users\elousiren\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\elousiren\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\elousiren\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [17]:
cat_f.append('card1-2')
cat_f.append('card1-5')
cat_f.append('card1-card5_addr1-2')
cat_check()

0:	test: 0.6247310	test1: 0.6497870	best: 0.6497870 (0)	total: 823ms	remaining: 13m 42s
50:	test: 0.9026794	test1: 0.8454265	best: 0.8466224 (47)	total: 33.7s	remaining: 10m 27s
100:	test: 0.9634089	test1: 0.8438340	best: 0.8473894 (53)	total: 1m 9s	remaining: 10m 18s
150:	test: 0.9718480	test1: 0.8507564	best: 0.8507564 (150)	total: 1m 45s	remaining: 9m 55s
200:	test: 0.9748814	test1: 0.8564833	best: 0.8565303 (193)	total: 2m 21s	remaining: 9m 23s
250:	test: 0.9783842	test1: 0.8610093	best: 0.8610093 (250)	total: 2m 57s	remaining: 8m 49s
300:	test: 0.9806713	test1: 0.8639718	best: 0.8639718 (300)	total: 3m 33s	remaining: 8m 15s
350:	test: 0.9825122	test1: 0.8659148	best: 0.8660362 (349)	total: 4m 8s	remaining: 7m 38s
400:	test: 0.9838527	test1: 0.8675708	best: 0.8675806 (398)	total: 4m 43s	remaining: 7m 2s
450:	test: 0.9845618	test1: 0.8685420	best: 0.8685502 (445)	total: 5m 17s	remaining: 6m 26s
500:	test: 0.9846959	test1: 0.8692515	best: 0.8692530 (499)	total: 5m 51s	remaining: 5m 4

###  Задание 3: Сделать FrequencyEncoder для признаков card1 - card6, addr1, addr2.

In [18]:
def freq_en(df, feature_list = ['card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2']):
    for feature in feature_list:
        freq_encoder = df[feature].value_counts(normalize=True)
        df[feature+"_freq_enc"] = df[feature].map(freq_encoder).astype('float64')

In [19]:
freq_en(x_train)
freq_en(x_valid)
freq_en(x_test)
freq_en(x_def)

C:\Users\elousiren\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [20]:
cat_check()

0:	test: 0.6604422	test1: 0.5775477	best: 0.5775477 (0)	total: 740ms	remaining: 12m 19s
50:	test: 0.9027438	test1: 0.8411043	best: 0.8411043 (50)	total: 33.6s	remaining: 10m 25s
100:	test: 0.9560183	test1: 0.8486813	best: 0.8488393 (97)	total: 1m 8s	remaining: 10m 5s
150:	test: 0.9653016	test1: 0.8527962	best: 0.8527962 (150)	total: 1m 44s	remaining: 9m 46s
200:	test: 0.9709006	test1: 0.8606645	best: 0.8606645 (200)	total: 2m 19s	remaining: 9m 15s
250:	test: 0.9739989	test1: 0.8628832	best: 0.8628832 (250)	total: 2m 55s	remaining: 8m 44s
300:	test: 0.9765532	test1: 0.8650828	best: 0.8653879 (291)	total: 3m 32s	remaining: 8m 13s
350:	test: 0.9790961	test1: 0.8663608	best: 0.8663682 (349)	total: 4m 8s	remaining: 7m 39s
400:	test: 0.9804016	test1: 0.8674733	best: 0.8674733 (400)	total: 4m 44s	remaining: 7m 4s
450:	test: 0.9811036	test1: 0.8683313	best: 0.8683313 (450)	total: 5m 17s	remaining: 6m 27s
500:	test: 0.9811367	test1: 0.8683570	best: 0.8683703 (482)	total: 5m 50s	remaining: 5m 49

### Задание 4: Создать признаки на основе отношения: TransactionAmt к вычисленной статистике. Статистика - среднее значение / стандартное отклонение TransactionAmt, сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

In [21]:
def create_numerical_aggs(data: pd.DataFrame,
                          groupby_id: str,
                          aggs: dict,
                          prefix: Optional[str] = None,
                          suffix: Optional[str] = None,
                          ) -> pd.DataFrame:
    """
    Построение агрегаций для числовых признаков.

    Parameters
    ----------
    data: pandas.core.frame.DataFrame
        Выборка для построения агрегаций.

    groupby_id: str
        Название ключа, по которому нужно произвести группировку.

    aggs: dict
        Словарь с названием признака и списка функций.
        Ключ словаря - название признака, который используется для
        вычисления агрегаций, значение словаря - список с названием
        функций для вычисления агрегаций.

    prefix: str, optional, default = None
        Префикс для названия признаков.
        Опциональный параметр, по умолчанию, не используется.

    suffix: str, optional, default = None
        Суффикс для названия признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    stats: pandas.core.frame.DataFrame
        Выборка с рассчитанными агрегациями.

    """
    if not prefix:
        prefix = ""
    if not suffix:
        suffix = "_groupby_"+groupby_id

    data_grouped = data.groupby(groupby_id)
    stats = data_grouped.agg(aggs)
    stats.columns = [f"{prefix}{feature}_{stat}{suffix}".lower() for feature, stat in stats]
    stats = stats.reset_index()

    return stats

In [22]:
def create_stats_feature(df, stats, groupby_id):
    df = df.merge(
        stats, how="left", on=groupby_id, suffixes=''
    )
    df["transactionamt_stats_groupby_"+groupby_id ] = df["transactionamt_mean_groupby_"+groupby_id] / df["transactionamt_std_groupby_"+groupby_id]
    df.drop(["transactionamt_mean_groupby_"+groupby_id, "transactionamt_std_groupby_"+groupby_id],axis=1)
    return df

In [23]:
groupby_id_list = ['card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'card1-2','card1-5','card1-card5_addr1-2']
aggs = {
    "TransactionAmt": [np.mean, np.std],
}

for groupby_id in groupby_id_list:
    stats = create_numerical_aggs(
        x_train, groupby_id=groupby_id, aggs=aggs
    )
    x_train = create_stats_feature(x_train, stats, groupby_id)
    x_valid = create_stats_feature(x_valid, stats, groupby_id)
    x_test = create_stats_feature(x_test, stats, groupby_id)
    x_def = create_stats_feature(x_def, stats, groupby_id)
    

C:\Users\elousiren\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:7959: FutureWarning: Passing 'suffixes' as a <class 'str'>, is not supported and may give unexpected results. Provide 'suffixes' as a tuple instead. In the future a 'TypeError' will be raised.
  validate=validate,


In [24]:
cat_check()

0:	test: 0.6038606	test1: 0.5883097	best: 0.5883097 (0)	total: 766ms	remaining: 12m 45s
50:	test: 0.9186582	test1: 0.8467068	best: 0.8469365 (48)	total: 33.7s	remaining: 10m 27s
100:	test: 0.9574477	test1: 0.8476694	best: 0.8478589 (73)	total: 1m 9s	remaining: 10m 15s
150:	test: 0.9616078	test1: 0.8512297	best: 0.8512297 (150)	total: 1m 44s	remaining: 9m 50s
200:	test: 0.9680771	test1: 0.8591529	best: 0.8591529 (200)	total: 2m 21s	remaining: 9m 22s
250:	test: 0.9739193	test1: 0.8638928	best: 0.8639380 (249)	total: 2m 58s	remaining: 8m 51s
300:	test: 0.9757593	test1: 0.8661954	best: 0.8661954 (300)	total: 3m 34s	remaining: 8m 18s
350:	test: 0.9795077	test1: 0.8684438	best: 0.8684633 (349)	total: 4m 10s	remaining: 7m 43s
400:	test: 0.9809974	test1: 0.8698485	best: 0.8699258 (399)	total: 4m 46s	remaining: 7m 8s
450:	test: 0.9818623	test1: 0.8705949	best: 0.8705996 (448)	total: 5m 22s	remaining: 6m 32s
500:	test: 0.9820607	test1: 0.8707078	best: 0.8707155 (494)	total: 5m 56s	remaining: 5m 

### Задание 5: Создать признаки на основе отношения: D15 к вычисленной статистике. Статистика - среднее значение / стандартное отклонение D15, сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

In [25]:
aggs = {
    "D15": [np.mean, np.std],
}

for groupby_id in groupby_id_list:
    stats = create_numerical_aggs(
        x_train, groupby_id=groupby_id, aggs=aggs
    )
    x_train = create_stats_feature(x_train, stats, groupby_id)
    x_valid = create_stats_feature(x_valid, stats, groupby_id)
    x_test = create_stats_feature(x_test, stats, groupby_id)
    x_def = create_stats_feature(x_def, stats, groupby_id)

C:\Users\elousiren\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:7959: FutureWarning: Passing 'suffixes' as a <class 'str'>, is not supported and may give unexpected results. Provide 'suffixes' as a tuple instead. In the future a 'TypeError' will be raised.
  validate=validate,


In [26]:
cat_check()

0:	test: 0.6926225	test1: 0.7268762	best: 0.7268762 (0)	total: 798ms	remaining: 13m 17s
50:	test: 0.9280167	test1: 0.8455763	best: 0.8455763 (50)	total: 34.5s	remaining: 10m 41s
100:	test: 0.9629663	test1: 0.8443201	best: 0.8463385 (55)	total: 1m 10s	remaining: 10m 30s
150:	test: 0.9682216	test1: 0.8482374	best: 0.8483175 (149)	total: 1m 47s	remaining: 10m 7s
200:	test: 0.9731530	test1: 0.8570072	best: 0.8570072 (200)	total: 2m 25s	remaining: 9m 37s
250:	test: 0.9761844	test1: 0.8624071	best: 0.8625279 (248)	total: 3m 1s	remaining: 9m 2s
300:	test: 0.9782096	test1: 0.8658896	best: 0.8659099 (299)	total: 3m 38s	remaining: 8m 27s
350:	test: 0.9796233	test1: 0.8677266	best: 0.8677652 (345)	total: 4m 14s	remaining: 7m 51s
400:	test: 0.9817584	test1: 0.8693385	best: 0.8694075 (397)	total: 4m 52s	remaining: 7m 16s
450:	test: 0.9826820	test1: 0.8705791	best: 0.8705814 (449)	total: 5m 28s	remaining: 6m 39s
500:	test: 0.9830543	test1: 0.8707381	best: 0.8707519 (480)	total: 6m 3s	remaining: 6m 1

### Задание 6: выделить дробную часть и целую часть признака TransactionAmt в два отдельных признака. После создать отдельных признак - логарифм от TransactionAmt

In [27]:
def delflo(df):
    df[['TransactionAmt_f', 'TransactionAmt_int']] = df['TransactionAmt'].apply(lambda x: math.modf(x)).apply(pd.Series)


In [28]:
delflo(x_train)
delflo(x_valid)
delflo(x_test)
delflo(x_def)

In [30]:
cat_check()

0:	test: 0.7067195	test1: 0.6919606	best: 0.6919606 (0)	total: 728ms	remaining: 12m 6s
50:	test: 0.8932423	test1: 0.8487822	best: 0.8487822 (50)	total: 33.9s	remaining: 10m 30s
100:	test: 0.9572482	test1: 0.8508438	best: 0.8525523 (57)	total: 1m 8s	remaining: 10m 7s
150:	test: 0.9622346	test1: 0.8539371	best: 0.8539371 (150)	total: 1m 44s	remaining: 9m 46s
200:	test: 0.9699215	test1: 0.8587681	best: 0.8587681 (200)	total: 2m 21s	remaining: 9m 22s
250:	test: 0.9732744	test1: 0.8620669	best: 0.8620669 (250)	total: 2m 58s	remaining: 8m 51s
300:	test: 0.9757463	test1: 0.8640314	best: 0.8640314 (300)	total: 3m 34s	remaining: 8m 18s
350:	test: 0.9770281	test1: 0.8660548	best: 0.8660548 (350)	total: 4m 10s	remaining: 7m 43s
400:	test: 0.9798392	test1: 0.8672627	best: 0.8674314 (389)	total: 4m 47s	remaining: 7m 8s
450:	test: 0.9817716	test1: 0.8678799	best: 0.8679672 (447)	total: 5m 22s	remaining: 6m 32s
500:	test: 0.9822704	test1: 0.8683348	best: 0.8683887 (473)	total: 5m 57s	remaining: 5m 56

### Задание 7 (опция): выполнить предварительную подготовку / очистку признаков P_emaildomain и R_emaildomain (что и как делать - остается на ваше усмотрение) и сделать Frequency Encoding для очищенных признаков.

In [31]:
def email_cl(df):
    df.loc[df['P_emaildomain']=='gmail', 'P_emaildomain'] = 'gmail.com'
    df['P_emaildomain'] = df['P_emaildomain'].str.replace(r'.mx$','')
    df.loc[df['R_emaildomain']=='gmail', 'R_emaildomain'] = 'gmail.com'
    df['R_emaildomain'] = df['R_emaildomain'].str.replace(r'.mx$','')    

In [32]:
x_train['P_emaildomain'].value_counts()

gmail.com           67198
yahoo.com           28992
missing             28440
hotmail.com         15058
anonymous.com       13328
aol.com              8583
comcast.net          2916
icloud.com           1677
outlook.com          1484
msn.com              1423
att.net              1223
sbcglobal.net        1092
verizon.net           971
live.com              894
bellsouth.net         713
ymail.com             654
me.com                624
cox.net               570
yahoo.com.mx          522
charter.net           372
optonline.net         362
live.com.mx           254
mac.com               214
rocketmail.com        201
earthlink.net         173
mail.com              147
outlook.es            142
roadrunner.com        119
gmail                 115
juno.com              111
embarqmail.com        109
windstream.net         95
twc.com                86
hotmail.es             85
frontier.com           79
q.com                  68
hotmail.fr             68
netzero.com            65
web.de      

In [33]:
x_train['R_emaildomain'].value_counts()

missing           119700
gmail.com          24644
hotmail.com        10129
anonymous.com       9882
yahoo.com           5666
                   ...  
netzero.net            7
scranton.edu           6
sc.rr.com              6
protonmail.com         5
hotmail.de             4
Name: R_emaildomain, Length: 61, dtype: int64

In [34]:
email_cl(x_train)
mask = x_train['P_emaildomain'].value_counts()
x_train.loc[x_train['P_emaildomain'].isin(mask.index[mask<500]), 'P_emaildomain'] = 'other'
mask = x_train.value_counts()

In [35]:
email_cl(x_valid)
email_cl(x_test)
email_cl(x_def)
x_valid.loc[x_valid['P_emaildomain'].isin(mask.index), 'P_emaildomain'] = 'other'
x_test.loc[x_test['P_emaildomain'].isin(mask.index), 'P_emaildomain'] = 'other'
x_def.loc[x_def['P_emaildomain'].isin(mask.index), 'P_emaildomain'] = 'other'

In [40]:
mask = x_train['R_emaildomain'].value_counts()
x_train.loc[x_train['R_emaildomain'].isin(mask.index[mask<200]), 'R_emaildomain'] = 'other'
mask = x_train.value_counts()

In [41]:
x_valid.loc[x_valid['R_emaildomain'].isin(mask.index), 'R_emaildomain'] = 'other'
x_test.loc[x_test['R_emaildomain'].isin(mask.index), 'R_emaildomain'] = 'other'
x_def.loc[x_def['R_emaildomain'].isin(mask.index), 'R_emaildomain'] = 'other'

In [42]:
freq_en(x_train, ['P_emaildomain', 'R_emaildomain'])
freq_en(x_valid, ['P_emaildomain', 'R_emaildomain'])
freq_en(x_test, ['P_emaildomain', 'R_emaildomain'])
freq_en(x_def, ['P_emaildomain', 'R_emaildomain'])

In [43]:
cat_check()

0:	test: 0.6599799	test1: 0.6615687	best: 0.6615687 (0)	total: 746ms	remaining: 12m 25s
50:	test: 0.8995976	test1: 0.8468652	best: 0.8475442 (45)	total: 34.2s	remaining: 10m 35s
100:	test: 0.9578212	test1: 0.8497331	best: 0.8506553 (66)	total: 1m 8s	remaining: 10m 13s
150:	test: 0.9683701	test1: 0.8519325	best: 0.8524231 (139)	total: 1m 46s	remaining: 9m 59s
200:	test: 0.9729862	test1: 0.8583981	best: 0.8583981 (200)	total: 2m 23s	remaining: 9m 30s
250:	test: 0.9765050	test1: 0.8609924	best: 0.8609924 (250)	total: 3m	remaining: 8m 59s
300:	test: 0.9782464	test1: 0.8642069	best: 0.8642069 (300)	total: 3m 37s	remaining: 8m 24s
350:	test: 0.9800308	test1: 0.8653033	best: 0.8654365 (335)	total: 4m 15s	remaining: 7m 52s
400:	test: 0.9808362	test1: 0.8669452	best: 0.8669452 (400)	total: 4m 52s	remaining: 7m 16s
450:	test: 0.9815070	test1: 0.8679516	best: 0.8679559 (449)	total: 5m 29s	remaining: 6m 41s
500:	test: 0.9817679	test1: 0.8679814	best: 0.8680355 (478)	total: 6m 3s	remaining: 6m 2s
5